In [5]:
# DOWNLOADING DATA

import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import multiprocessing
from joblib import Parallel, delayed
from tqdm import tqdm
from sklearn import metrics
import pprint
import copy
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly import offline
import statsmodels.graphics.tsaplots as tsa
import warnings
warnings.filterwarnings("ignore")

tomato_discount = pd.read_csv('Dataframes/tomato_discount.csv')
tomato_7 = pd.read_csv('Dataframes/tomato_7.csv')

# tomato_7

In [6]:
# CALCULATING CLEAR DEMAND

IDs = [col.strip('is_discount_') for col in tomato_discount.columns if 'is_discount_' in col]
ZIP_non_price_models =[]
MSE = []

for id in IDs:
    endog = tomato_7['salesvolume_' + id]
    exog = tomato_7[['av_price_' + id, 'is_supplied_' + id]]
    ZIP_non_price_models.append(
        sm.ZeroInflatedPoisson(endog = endog, exog = exog, exog_infl = exog, inflation = 'logit').fit(maxiter = 120))
    non_price_demand = ZIP_non_price_models[IDs.index(id)].predict(exog, exog_infl = exog)

    tomato_7['non_price_demand_' + id] = tomato_7['salesvolume_' + id] - non_price_demand

    MSE.append(metrics.mean_squared_error(tomato_7['salesvolume_' + id], non_price_demand))

print(f'TOTAL MSE: {sum(MSE)}')

Optimization terminated successfully.
         Current function value: 1.881987
         Iterations: 32
         Function evaluations: 39
         Gradient evaluations: 39
Optimization terminated successfully.
         Current function value: 1.164183
         Iterations: 34
         Function evaluations: 38
         Gradient evaluations: 38
Optimization terminated successfully.
         Current function value: 1.158541
         Iterations: 42
         Function evaluations: 48
         Gradient evaluations: 48
Optimization terminated successfully.
         Current function value: 1.048035
         Iterations: 16
         Function evaluations: 21
         Gradient evaluations: 21
Optimization terminated successfully.
         Current function value: 2.725490
         Iterations: 28
         Function evaluations: 33
         Gradient evaluations: 33
Optimization terminated successfully.
         Current function value: 1.210461
         Iterations: 31
         Function evaluations: 35
  

In [7]:
# SETTING Xs, ys, IDs

lags_on_Xs = []
Xs = []; ys = []

def sum_of_last(series, n = 7):
    ser_list = []

    for i in range(0, n):
        ser_list.append(0)

    for i in range(n, len(series)):
        ser_list.append(np.sum([j for j in series[(i - n):i]]))

    return ser_list

for id in IDs:
    tomato_7_process = tomato_7.copy()

    # adding num_other_discounts_
    tomato_7_process = tomato_7_process.merge(tomato_discount[['date', 'num_other_discounts_' + id]], on = 'date')

    # rename target columns
    tomato_7_process.rename(columns = {'salesvolume_' + id: 'target',
                                        'non_price_demand_' + id: 'tar_non_price',
                                        'av_price_' + id:'tar_price',
                                        'num_other_discounts_' + id: 'tar_other_discounts',
                                        'is_supplied_' + id: 'tar_is_supp'}, inplace = True)

    # drop salesvolume_
    to_drop = [col for col in tomato_7_process.columns if 'salesvolume_' in col]   # 'non_price_demand_'
    tomato_7_process.drop(to_drop, axis = 1, inplace = True)

    # shifting back of non_price_demand_
    work_with = [col for col in tomato_7_process.columns if 'non_price_demand_' in col]    # 'salesvolume_'
    tomato_7_process.loc[:, work_with] = tomato_7_process.loc[:, work_with].shift(-1)
    tomato_7_process = tomato_7_process.iloc[:-1, :].reset_index(drop = True)

    for col in work_with:
        tomato_7_process.rename(columns = {col: col + '_lag'}, inplace = True)

    # prices only when is supplied
    to_supp = [col for col in tomato_7_process.columns if 'av_price_' in col]
    for col in to_supp:
        tomato_7_process[col] = tomato_7_process[col] * tomato_7_process['is_supplied_' + col.replace('av_price_', '')]

    # adding lags on target variables
    lags_on_X = [[*tsa.pacf(tomato_7_process['target'])].index(sorted([*tsa.pacf(tomato_7_process['target'])])[::-1][lag]) for lag in [1, 2, 3]]
    lags_on_Xs.append(lags_on_X.copy())

    for i in lags_on_X:
        for col in ['tar_non_price', 'tar_price']:
            tomato_7_process[col + '_' + str(i)] = tomato_7_process[col].shift(i)

    tomato_7_process = tomato_7_process.iloc[10:, :].reset_index(drop = True)

    # adding week statistics
    tomato_7_process['tar_non_price_week'] = sum_of_last(tomato_7_process['tar_non_price'], n = 7)
    tomato_7_process['non_price_demand_week'] = \
        sum_of_last(tomato_7_process[[col for col in tomato_7_process.columns if 'non_price_demand_' in col]].sum(axis = 1), n = 7)
    tomato_7_process = tomato_7_process.iloc[7:, :].reset_index(drop = True)

    # changing the order
    tomato_7_process = tomato_7_process\
                            .loc[:, [c for c in tomato_7_process.columns if 'tar' in c] +
                                    [c for c in tomato_7_process.columns if 'tar' not in c]]

    # setting X and y
    Xs.append(tomato_7_process.drop(['target', 'tar_non_price', 'date'], axis = 1).copy())
    ys.append(tomato_7_process['target'].copy())

    # setting date
    if id == IDs[0]:
        date = tomato_7_process['date']

In [8]:
def timesplit(X, y, train_size = 0.85):
    n_train = int(len(y) * train_size)
    return X[:n_train], X[n_train:], y[:n_train], y[n_train:]

X_trains = []; X_tests = []; y_trains = []; y_tests = []

for i in range(0, len(Xs)):
    X_train, X_test, y_train, y_test = timesplit(Xs[i], ys[i])
    X_trains.append(X_train.copy()); X_tests.append(X_test.copy()); y_trains.append(y_train.copy()); y_tests.append(y_test.copy())

In [9]:
# FITTING ZERO INFLATED POISSON MODELS

# https://towardsdatascience.com/negative-binomial-regression-f99031bb25b4

def NB2_fitting_i(X_trains, y_trains, i):
    X_in_for = pd.DataFrame()

    def block(y, X):
        Poisson_reg = sm.GLM(y, X, family = sm.families.Poisson())\
            .fit()

        df_train = pd.concat([y, X], join = 'outer', axis = 1)
        df_train['BB_LAMBDA'] = Poisson_reg.mu
        df_train['AUX_OLS_DEP'] = df_train.apply(lambda x: ((x['target'] - x['BB_LAMBDA']) ** 2 - x['BB_LAMBDA']) / x['BB_LAMBDA'], axis = 1)

        aux_olsr_results = smf.ols("""AUX_OLS_DEP ~ BB_LAMBDA - 1""", df_train)\
            .fit()
        alpha = aux_olsr_results.params[0] if aux_olsr_results.params[0] > 0 else 1 / 10 ** 4

        return alpha

    for col_num in range(0, X_trains[i].shape[1]):
        X_in_for = pd.concat([X_in_for, X_trains[i].iloc[:, col_num]], axis = 1)

        for p_value in [0.5, 0.2, 0.1]:
            NB2_reg = sm.GLM(y_trains[i], X_in_for, family = sm.families.NegativeBinomial(alpha = block(y = y_trains[i], X = X_in_for)))\
            .fit()

            signif_var = pd.DataFrame([x for x in NB2_reg.summary().tables[1].data[1:] if float(x[4]) < p_value],
                                  columns = NB2_reg.summary().tables[1].data[0])

            if len([*signif_var.iloc[:, 0]]) > 0:
                X_in_for = X_in_for.loc[:, [*signif_var.iloc[:, 0]]]

    k = 1
    while k == 1:
        p_values_of_NB2 = [float(x) for x in [*signif_var.iloc[:, 4]]]
        max_p_value = max(p_values_of_NB2)

        if max_p_value > 0.00001:
            signif_vars_for_X = [*signif_var.iloc[:, 0]]
            signif_vars_for_X.pop(p_values_of_NB2.index(max_p_value))

            X_in_for = X_in_for.loc[:, signif_vars_for_X]

            NB2_reg = sm.GLM(y_trains[i], X_in_for, family = sm.families.NegativeBinomial(alpha = block(y = y_trains[i], X = X_in_for)))\
                .fit()

            signif_var = pd.DataFrame([x for x in NB2_reg.summary().tables[1].data[1:]],
                                  columns = NB2_reg.summary().tables[1].data[0])
        else: k = 0

    return [*signif_var.iloc[:, 0]]

NB2_signif_vars = Parallel(n_jobs = multiprocessing.cpu_count() - 3)\
    (delayed(NB2_fitting_i)(X_trains = X_trains, y_trains = y_trains, i = i) for i in tqdm(range(0, len(X_trains))))

for j in [14, 15]:
    NB2_signif_vars[j].remove('is_supplied_461504')
    NB2_signif_vars[j].append('tar_is_supp')

ZIPmodels = []
for i in range(0, len(Xs)):
    ZIPmodels.append(
        sm.ZeroInflatedPoisson(endog = y_trains[i],
                               exog = X_trains[i][NB2_signif_vars[i]],
                               exog_infl = X_trains[i][NB2_signif_vars[i]],
                               inflation = 'logit').fit(maxiter = 100))

ZIP_train_preds = [ZIPmodels[i].predict(X_trains[i][NB2_signif_vars[i]], exog_infl = X_trains[i][NB2_signif_vars[i]])  for i  in range(0, len(Xs))]

ZIP_test_preds = [ZIPmodels[i].predict(X_tests[i][NB2_signif_vars[i]], exog_infl = X_tests[i][NB2_signif_vars[i]])  for i  in range(0, len(Xs))]

pd.DataFrame([np.array(j) for j in ZIP_train_preds]).T\
    .to_csv('Results/Vector models/Vector ZIP/ZIP_train_preds.csv', index = False)
pd.DataFrame([np.array(j) for j in ZIP_test_preds]).T\
    .to_csv('Results/Vector models/Vector ZIP/ZIP_test_preds.csv', index = False)

MSE_trains = [metrics.mean_squared_error(y_trains[i], ZIP_train_preds[i]) for i in range(0, len(Xs))]
print(f'MSE_trains_sum: {sum(MSE_trains)}')
MSE_tests = [metrics.mean_squared_error(y_tests[i], ZIP_test_preds[i]) for i in range(0, len(Xs))]
print(f'MSE_tests_sum: {sum(MSE_tests)}')

100%|██████████| 20/20 [03:26<00:00, 10.33s/it]


         Current function value: 1.569688
         Iterations: 100
         Function evaluations: 112
         Gradient evaluations: 112
         Current function value: 1.192725
         Iterations: 100
         Function evaluations: 124
         Gradient evaluations: 124
         Current function value: 1.139817
         Iterations: 100
         Function evaluations: 110
         Gradient evaluations: 110
         Current function value: 0.893395
         Iterations: 100
         Function evaluations: 108
         Gradient evaluations: 108
         Current function value: 1.944551
         Iterations: 100
         Function evaluations: 112
         Gradient evaluations: 112
         Current function value: 1.303086
         Iterations: 100
         Function evaluations: 117
         Gradient evaluations: 117
         Current function value: 1.257327
         Iterations: 100
         Function evaluations: 113
         Gradient evaluations: 113
Optimization terminated successfully.
   

In [7]:
# FORECASTING WITH ZERO INFLATED POISSON MODELS

def vector_ZIP_forecast(ZIPmodels, Xs, ys, lags:list):
    global IDs, vector_ZIP_forecast_row
    Xs = copy.deepcopy(Xs)
    ys = copy.deepcopy(ys)
    forecast_lists = [[] for l in range(0, len(Xs))]

    def vector_ZIP_forecast_row(i):
        nonlocal Xs, ys, lags, forecast_lists, row
        global ZIPmodels, IDs

        # block of code without lags
        for id in [id for id in IDs if id != IDs[i]]:
            if row >= 1:
                Xs[i].loc[Xs[i].index[0] + row, 'non_price_demand_' + str(id) + '_lag'] = \
                    forecast_lists[IDs.index(id)][row - 1] - \
                    ZIP_non_price_models[IDs.index(id)].predict(
                        pd.DataFrame([Xs[IDs.index(id)].loc[Xs[IDs.index(id)].index[0] + row - 1, ['tar_price', 'tar_is_supp']]], columns=['tar_price', 'tar_is_supp'])\
                      .rename(columns={'tar_price': 'av_price_' + id, 'tar_is_supp': 'is_supplied_' + id})\
                            .reset_index(drop = True),
                    exog_infl =
                        pd.DataFrame([Xs[IDs.index(id)].loc[Xs[IDs.index(id)].index[0] + row - 1, ['tar_price', 'tar_is_supp']]], columns=['tar_price', 'tar_is_supp'])\
                      .rename(columns={'tar_price': 'av_price_' + id, 'tar_is_supp': 'is_supplied_' + id})\
                            .reset_index(drop = True))[0]

        #!!!!!!!!!!!!!!!!!!!!!!!
        if row >= 7:
            Xs[i].loc[Xs[i].index[0] + row, 'tar_non_price_week'] = \
                np.sum([t for t in Xs[i].loc[(Xs[i].index[0] + row - 6):(Xs[i].index[0] + row), 'tar_non_price_1']])

            Xs[i].loc[Xs[i].index[0] + row, 'non_price_demand_week'] = \
                sum(Xs[i].loc[
                (Xs[i].index[0] + row - 6):(Xs[i].index[0] + row),
                [c for c in Xs[i].columns if 'non_price_demand' in c and 'week' not in c]].sum(axis = 1))

        # block of code with lags
        for lag in lags[i]:
            if row >= lag:
                Xs[i].loc[Xs[i].index[0] + row, 'tar_non_price_' + str(lag)] = \
                    forecast_lists[i][row - lag] - \
                    ZIP_non_price_models[i].predict(
                        pd.DataFrame([Xs[i].loc[Xs[i].index[0] + row - lag, ['tar_price', 'tar_is_supp']]], columns=['tar_price', 'tar_is_supp'])\
                      .rename(columns={'tar_price': 'av_price_' + IDs[i], 'tar_is_supp': 'is_supplied_' + IDs[i]})\
                            .reset_index(drop = True),
                    exog_infl =
                        pd.DataFrame([Xs[i].loc[Xs[i].index[0] + row - lag, ['tar_price', 'tar_is_supp']]], columns=['tar_price', 'tar_is_supp'])\
                      .rename(columns={'tar_price': 'av_price_' + IDs[i], 'tar_is_supp': 'is_supplied_' + IDs[i]})\
                            .reset_index(drop = True))[0]

        forecast_lists[i].append(ZIPmodels[i].predict(
            pd.DataFrame([Xs[i].loc[Xs[i].index[0] + row, :][NB2_signif_vars[i]]], columns = NB2_signif_vars[i]).reset_index(drop = True),
            exog_infl = pd.DataFrame([Xs[i].loc[Xs[i].index[0] + row, :][NB2_signif_vars[i]]], columns = NB2_signif_vars[i]).reset_index(drop = True)
        )[0])

        return

    for row in tqdm(range(0, len(ys[0]))):
        Parallel(n_jobs = 1)\
            (delayed(vector_ZIP_forecast_row)(i = i) for i in range(0, len(Xs))) # n_jobs = multiprocessing.cpu_count() - 1

    result = pd.DataFrame([np.array(j) for j in forecast_lists]).T
    result.columns = ['salesvolume_' + id for id in IDs]
    return result, Xs

ZIP_test_forecasts, Xs_forecast = vector_ZIP_forecast(ZIPmodels = ZIPmodels, Xs = X_tests, ys = y_tests, lags = lags_on_Xs)

ZIP_test_forecasts.to_csv('Results/Vector models/Vector ZIP/ZIP_test_forecasts.csv', index = False)

MSE_forecast = [metrics.mean_squared_error(y_tests[i], ZIP_test_forecasts.iloc[:, i]) for i in range(0, len(Xs))]
print(f'MSE_tests_sum: {sum(MSE_forecast)}')

100%|██████████| 326/326 [07:50<00:00,  1.44s/it]


MSE_tests_sum: 127.3605544450175


In [10]:
train_preds = pd.read_csv('Results/Vector models/Vector ZIP/ZIP_train_preds.csv')
test_preds = pd.read_csv('Results/Vector models/Vector ZIP/ZIP_test_preds.csv')
test_forecasts = pd.read_csv('Results/Vector models/Vector ZIP/ZIP_test_forecasts.csv')

In [15]:
# GRAPH GBDT of ID

def graph_juice(number = None, id = None):
    number = number if number is not None else IDs.index(id)

    fig = make_subplots(rows = 3, cols = 1)

    for row, what in zip([1, 2, 3], ['train_pred', 'test_pred', 'test_forecast']):
        y_actual = y_tests[number] if what in ['test_pred', 'test_forecast'] else y_trains[number]

        if what == 'train_pred':
            y_pred = train_preds.iloc[:, number]
        elif what == 'test_pred':
            y_pred = test_preds.iloc[:, number]
        elif what == 'test_forecast':
            y_pred = test_forecasts.iloc[:, number]

        fig.add_trace(go.Scatter(x = date[y_actual.index],
                                 y = y_actual,
                                 mode = 'lines+markers',
                                 name = 'Actual counts',
                                 marker = dict(color = '#00A383', size = 3.5),
                                 line = dict(color = '#00A383', width = 1.5)),
                      row = row, col = 1)

        fig.add_trace(go.Scatter(x = date[y_actual.index],
                                 y =  y_pred,
                                 # y =  [*map(round, y_pred)],
                                 mode = 'lines',
                                 name = 'Predicted counts',
                                 line = dict(color = '#F53D65', width = 2.5)),
                      row = row, col = 1)

        fig.add_trace(go.Scatter(x = date[y_actual.index],
                                 y = tomato_discount[tomato_discount['date'].isin(date[y_actual.index])]\
                                 .loc[:, 'is_discount_' + IDs[number]].replace(0, np.NaN) *\
                                 (y_actual.max() / tomato_discount[tomato_discount['date'].isin(date[y_actual.index])]\
                                 .loc[:, 'is_discount_' + IDs[number]].max() / 1.5),
                                 mode = 'lines',
                                 name = 'Presence of discount',
                                 line = dict(color = '#B1F100', width = 5)),
                      row = row, col = 1)

    fig.update_layout(height = 1000, width = 1100,
                      title_text = f'Gradient boost of decision trees - train, test predictions, test forecast - {IDs[number]} sku',
                      showlegend = False)
    # offline.plot(fig, filename='file.html')
    fig.show()
# 6, 8, 14 !, 15 !
graph_juice(number = 16)